In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from google.colab import drive
import copy
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_conv = models.alexnet(pretrained = True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [3]:
for param in model_conv.parameters():
  param.requires_grad = False

model_conv.classifier[4] = nn.Linear(4096,1024)
model_conv.classifier[6] = nn.Linear(1024,3)
model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.01, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=30, gamma=0.1) 

In [11]:

data_dir = '/content/gdrive/My Drive/main_dir' 
data_transforms = {
 'train': transforms.Compose([
 # put the input to Tensor format in order to use torch
    transforms.ToTensor() 
 #transforms.Normalize( ?? ) 
 ]),
 'valid': transforms.Compose([
    transforms.ToTensor()
   ]),                        
 'test': transforms.Compose([
    transforms.ToTensor()                       
   ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
 data_transforms[x]) for x in ['train', 'valid', 'test']}

loader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 1, shuffle = True, num_workers = 4) for x in ['train', 'test', 'valid']}
#loader = torch.utils.data.DataLoader(train_set, batch_size=len(train_set), num_workers=1)
#data = next(iter(loader))
#data[0].mean(), data[0].std()
#data = next(iter(loader))
#data[0].mean(), data[0].std()
mean = 0.
std = 0.
mean_t = 0.
std_t = 0.
mean_v = 0.
std_v = 0.
for images, _ in loader['train']:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)

for images, _ in loader['test']:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean_t += images.mean(2).sum(0)
    std_t += images.std(2).sum(0)

for images, _ in loader['valid']:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean_v += images.mean(2).sum(0)
    std_v += images.std(2).sum(0)

mean /= len(loader['train'].dataset)
mean_t /= len(loader['test'].dataset)
mean_v /= len(loader['valid'].dataset)
std /= len(loader['train'].dataset)
std_t /= len(loader['test'].dataset)
std_v /= len(loader['valid'].dataset)

data_transforms = {
 'train': transforms.Compose([
 # put the input to Tensor format in order to use torch
    transforms.ToTensor(),
    #transforms.Normalize(mean, std) uncomment 3 lines with normalization to use normalization 
 ]),
 'valid': transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean_v, std_v)
   ]),                        
 'test': transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean_t, std_t)                      
   ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
 data_transforms[x]) for x in ['train', 'valid', 'test']}

mean, mean_t, mean_v, std, std_t, std_v
image_datasets

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


{'test': Dataset ImageFolder
     Number of datapoints: 144
     Root location: /content/gdrive/My Drive/main_dir/test
     StandardTransform
 Transform: Compose(
                ToTensor()
            ), 'train': Dataset ImageFolder
     Number of datapoints: 150
     Root location: /content/gdrive/My Drive/main_dir/train
     StandardTransform
 Transform: Compose(
                ToTensor()
            ), 'valid': Dataset ImageFolder
     Number of datapoints: 36
     Root location: /content/gdrive/My Drive/main_dir/valid
     StandardTransform
 Transform: Compose(
                ToTensor()
            )}

In [12]:
image_datasets

{'test': Dataset ImageFolder
     Number of datapoints: 144
     Root location: /content/gdrive/My Drive/main_dir/test
     StandardTransform
 Transform: Compose(
                ToTensor()
            ), 'train': Dataset ImageFolder
     Number of datapoints: 150
     Root location: /content/gdrive/My Drive/main_dir/train
     StandardTransform
 Transform: Compose(
                ToTensor()
            ), 'valid': Dataset ImageFolder
     Number of datapoints: 36
     Root location: /content/gdrive/My Drive/main_dir/valid
     StandardTransform
 Transform: Compose(
                ToTensor()
            )}

In [13]:
def train_model(image_datasets, model, criterion, optimizer, scheduler, num_epochs):
  dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, shuffle = True, num_workers = 4) for x in ['train', 'valid']}
  best_model_wts = copy.deepcopy(model.state_dict())
  best_no_corrects = 0

  for epoch in range(num_epochs):
    print(epoch)
 # Set the model to the training mode for updating the weights using 
 # the first portion of training images
    model.train() 
    for inputs, labels in dataloaders['train']: # iterate over data
      inputs = inputs.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 # Set the model to the evaluation mode for selecting the best network 
 # based on the number of correctly classified validation images
    model.eval()
    no_corrects = 0
    for inputs, labels in dataloaders['valid']:
      inputs = inputs.to(device)
      labels = labels.to(device)
      with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        no_corrects += torch.sum(preds == labels.data)
    if no_corrects > best_no_corrects:
      best_no_corrects = no_corrects
      best_model_wts = copy.deepcopy(model.state_dict())
    scheduler.step()
 # Load the weights of the best network
  model.load_state_dict(best_model_wts)
  return model

In [14]:
model1 = train_model(image_datasets, model_conv, criterion, optimizer_conv, exp_lr_scheduler, 100)

0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
model1

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [15]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 1, shuffle = True, num_workers = 4) for x in ['train', 'test', 'valid']}

correct_train = [0, 0, 0]
incorrect_train = [0, 0, 0]

for inputs, labels in dataloaders['train']:
  inputs = inputs.to(device)
  labels = labels.to(device)
  with torch.set_grad_enabled(False):
    outputs = model1(inputs)
    _, preds = torch.max(outputs, 1)
    if preds == labels.data:
      correct_train[preds[0]]+=1
    else:
      incorrect_train[preds[0]]+=1

correct_test = [0, 0, 0]
incorrect_test = [0, 0, 0]

for inputs, labels in dataloaders['test']:
  inputs = inputs.to(device)
  labels = labels.to(device)
  with torch.set_grad_enabled(False):
    outputs = model1(inputs)
    _, preds = torch.max(outputs, 1)
    if preds == labels.data:
      correct_test[preds[0]]+=1
    else:
      incorrect_test[preds[0]]+=1

correct_valid = [0, 0, 0]
incorrect_valid = [0, 0, 0]

for inputs, labels in dataloaders['valid']:
  inputs = inputs.to(device)
  labels = labels.to(device)
  with torch.set_grad_enabled(False):
    outputs = model1(inputs)
    _, preds = torch.max(outputs, 1)
    if preds == labels.data:
      correct_valid[preds[0]]+=1
    else:
      incorrect_valid[preds[0]]+=1



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
correct_train, incorrect_train, correct_test, incorrect_test, correct_valid, incorrect_valid
acc_train = [0, 0 ,0]
acc_test = [0, 0 ,0]
acc_valid = [0, 0 ,0]
for i in range(3):
  acc_train[i] = correct_train[i] / (correct_train[i] + incorrect_train[i])
  acc_test[i] = correct_test[i] / (correct_test[i] + incorrect_test[i])
  acc_valid[i] = correct_valid[i] / (correct_valid[i] + incorrect_valid[i])
acc_train, acc_test, acc_valid

([0.7391304347826086, 0.7701149425287356, 0.9411764705882353],
 [0.7872340425531915, 0.7297297297297297, 0.8695652173913043],
 [0.6153846153846154, 0.8095238095238095, 1.0])